In [6]:
import os
import string
from string import Template

class mt(Template):
    delimiter = '&'
    idpattern = r'[a-z][_a-z0-9]*'

class hardware_exp:
    def __init__(self, acc_name, acc_version, acc_sub_version, acc_test, acc_test_id,acc_test_desc,acc_link_folder, vivado_path):
        self.acc_name = acc_name
        self.acc_version = acc_version
        self.acc_sub_version = acc_sub_version
        self.acc_test = acc_test
        self.acc_test_id = ("_"+ self.acc_test_id) if self.acc_test else ""
        self.acc_test_desc = acc_test_desc
        self.acc_link_folder = acc_link_folder + acc_name +"_src"
        self.acc_tag = str(self.acc_name) + "_" + str(self.acc_version) + "_" + str(self.acc_sub_version) + str(self.acc_test_id)
        self.acc_part = "xc7z020clg400-1"
        self.vp = vivado_path

    def generate_hls_tcl(self, top,output_dir):
        s = ""
        s += "open_project -reset " + self.acc_tag + "\n"
        s += "set_top " + top + "\n"
        for file in os.listdir(self.acc_link_folder):
            if file.endswith(".cc") or file.endswith(".h"):
                s += "add_files " + "src/" + file + ' -cflags "-D__SYNTHESIS__"\n'
        
        # for file in os.listdir(self.acc_link_folder):
        #     if file.endswith(".h"):
        #         s += "add_files " + "src/" + file + ' -cflags "-D__SYNTHESIS__"\n'

        s += 'open_solution "' + self.acc_tag + '"\n'
        s += "set_part " + self.acc_part + "\n"
        s += "create_clock -period " + "5" + " -name default\n"
        s += "config_export -format ip_catalog -rtl verilog -taxonomy /s -vendor xilinx\n"
        s += "csynth_design\n"
        s += "export_design -format ip_catalog\n"
        s += "exit\n"

        with open(f"{output_dir}/hls_script.tcl", "w") as f:
            f.write(s)

    def generate_hlx_tcl(self, output_dir, hlx_tcl_script,top):
        with open((hlx_tcl_script)) as f:
            tcl_script = str(mt(f.read()).substitute({"top":top}))
        with open(f"{output_dir}/hlx_script.tcl", "w") as f:
            f.write(tcl_script)

    def create_run_script(self, output_path, hlx_tcl_script):
        output_dir = output_path + "/" + self.acc_tag
        run_script = "#!/bin/bash\n"
        run_script += f"rm -rf ./{self.acc_tag}\n"
        run_script += f"rm -rf ./{self.acc_tag}_hlx\n"
        run_script += f"rm -rf ./Xil\n"
        run_script += f"rm -rf ./generated_files\n"
        run_script += f"rm -rf ./NA\n"
        run_script += f"rm -rf ./vivado*\n"

        run_script += "start_hls=`date +%s.%N`\n"
        run_script += f"{self.vp}/vivado_hls -f hls_script.tcl\n"
        run_script += "end_hls=`date +%s.%N`\n"
        run_script += f"{self.vp}/vivado -mode batch -source hlx_script.tcl -tclargs --origin_dir . --project_name {self.acc_tag}_hlx --ip_repo ./{self.acc_tag}/{self.acc_tag}\n"
        run_script += "end_hlx=`date +%s.%N`\n"
        run_script += "mkdir -p ./generated_files\n"
        run_script += f"cp ./{self.acc_tag}_hlx/{self.acc_tag}_hlx.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh ./generated_files/{self.acc_tag}.hwh\n"
        run_script += f"cp ./{self.acc_tag}_hlx/{self.acc_tag}_hlx.runs/impl_1/design_1_wrapper.bit ./generated_files/{self.acc_tag}.bit\n"
        run_script += f"cp ./{self.acc_tag}_hlx/utilization_report_impl_full.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/utilization_report_impl_ip.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/timing_report_impl_full.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/timing_report_impl_ip.txt ./generated_files/\n"
        run_script += f"hls_runtime=$((start_hls-end_hls))\n"
        run_script += f"hlx_runtime=$((end_hlx-end_hls))\n"
        run_script += f"echo HLS runtime: $hls_runtime\n"
        run_script += f"echo HLX runtime: $hlx_runtime\n"

        with open(f"{output_dir}/run.sh", "w") as f:
            f.write(run_script)
        os.system(f"chmod +x {output_dir}/run.sh")
        
    def create_project(self, top, output_path, hlx_tcl_script):
        output_dir = output_path + "/" + self.acc_tag
        output_acc_src_dir = output_dir + "/src"
        os.system(f"mkdir -p {output_dir}")

        # Copy .h and .cc files from the link folder to the output folder
        os.system(f"mkdir -p {output_acc_src_dir}")
        os.system(f"cp {self.acc_link_folder}/*.h {output_acc_src_dir}")
        os.system(f"cp {self.acc_link_folder}/*.cc {output_acc_src_dir}")
        self.generate_hls_tcl(top,output_dir)
        self.generate_hlx_tcl(output_dir, hlx_tcl_script,top)
        self.create_run_script(output_path, hlx_tcl_script)

  

In [5]:
# VMv2
acc_name = "vm"
acc_version = 2
acc_sub_version = 0
acc_test = False
acc_test_id = 0
acc_test_desc = "test_0"
acc_link_folder = "./acc_srcs/"
vivado_path = "/home/jude/Xilinx_2019.2/Vivado/2019.2/bin/"
hlx_tcl_script = "./hlx_scripts/vm_hlx.tcl"
h1 = hardware_exp(acc_name, acc_version, acc_sub_version, acc_test, acc_test_id,acc_test_desc,acc_link_folder, vivado_path)
h1.create_project("VM_INT8_V2_0", "generated", hlx_tcl_script)

In [7]:
# MM2IMv1
acc_name = "mm2im"
acc_version = 1
acc_sub_version = 0
acc_test = False
acc_test_id = 0
acc_test_desc = "test_0"
acc_link_folder = "./acc_srcs/"
vivado_path = "/home/jude/Xilinx_2019.2/Vivado/2019.2/bin/"
hlx_tcl_script = "./hlx_scripts/mm2im_hlx.tcl"
h1 = hardware_exp(acc_name, acc_version, acc_sub_version, acc_test, acc_test_id,acc_test_desc,acc_link_folder, vivado_path)
h1.create_project("MM2IMv1", "generated", hlx_tcl_script)